In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score
import sys
import os

# --- 1. LOAD PREDICTIONS ---
print("Loading model predictions...")
df = pd.read_csv('../data/processed/05_model_predictions.csv', index_col='DateTime', parse_dates=True)

print(f"Loaded {len(df)} predictions\n")

# --- 2. PREPARE TARGETS & PREDICTIONS ---
y_true = df['Target_Direction'].values
y_true_binary = (y_true > 0.5).astype(int)

models = ['Ridge', 'LSTM']
predictions = {}

for model in models:
    # Map model names to column names
    if model == 'Ridge':
        col_name = 'Ridge_Pred'
    elif model == 'LSTM':
        col_name = 'LSTM_Pred'
    
    predictions[model] = {
        'raw': df[col_name].values,
        'binary': (df[col_name].values > 0.5).astype(int)
    }

# --- 3. CALCULATE COMPREHENSIVE METRICS ---
print("="*70)
print("MODEL EFFICIENCY METRICS")
print("="*70 + "\n")

metrics_data = {
    'Model': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1-Score': [],
    'ROC-AUC': []
}

for model in models:
    y_pred = predictions[model]['binary']
    y_pred_proba = predictions[model]['raw']
    
    # Calculate metrics
    accuracy = accuracy_score(y_true_binary, y_pred)
    precision = precision_score(y_true_binary, y_pred, zero_division=0)
    recall = recall_score(y_true_binary, y_pred, zero_division=0)
    f1 = f1_score(y_true_binary, y_pred, zero_division=0)
    
    # ROC-AUC requires probability predictions
    try:
        roc_auc = roc_auc_score(y_true_binary, y_pred_proba)
    except:
        roc_auc = np.nan
    
    metrics_data['Model'].append(model)
    metrics_data['Accuracy'].append(accuracy)
    metrics_data['Precision'].append(precision)
    metrics_data['Recall'].append(recall)
    metrics_data['F1-Score'].append(f1)
    metrics_data['ROC-AUC'].append(roc_auc)

# Create DataFrame
metrics_df = pd.DataFrame(metrics_data).set_index('Model')

# Display with formatting
print(metrics_df.round(4).to_string())
print("\n" + "="*70 + "\n")

# --- 4. RANKING & SUMMARY STATISTICS ---
print("RANKINGS BY METRIC:\n")

for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']:
    ranking = metrics_df[metric].sort_values(ascending=False)
    print(f"{metric}:")
    for i, (model, score) in enumerate(ranking.items(), 1):
        print(f"  {i}. {model:18s} → {score:.4f}")
    print()

# --- 5. OVERALL WINNER ---
print("="*70)

# Calculate average rank across metrics
rank_scores = pd.DataFrame()
for metric in ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']:
    rank_scores[metric] = metrics_df[metric].rank(ascending=False)

rank_scores['Average Rank'] = rank_scores.mean(axis=1)
best_model = rank_scores['Average Rank'].idxmin()

print(f"\n🏆 Best Overall Model (Average Rank): {best_model}")
print(f"\n   Metrics:")
print(f"   - Accuracy:  {metrics_df.loc[best_model, 'Accuracy']:.4f}")
print(f"   - Precision: {metrics_df.loc[best_model, 'Precision']:.4f}")
print(f"   - Recall:    {metrics_df.loc[best_model, 'Recall']:.4f}")
print(f"   - F1-Score:  {metrics_df.loc[best_model, 'F1-Score']:.4f}")
print(f"   - ROC-AUC:   {metrics_df.loc[best_model, 'ROC-AUC']:.4f}")

print("\n" + "="*70)

# --- 6. MODEL COMPARISON ---
print("\nMODEL COMPARISON:\n")

ridge_metrics = metrics_df.loc['Ridge']
lstm_metrics = metrics_df.loc['LSTM']

comparison_data = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC'],
    'Ridge': [ridge_metrics['Accuracy'], ridge_metrics['Precision'], 
              ridge_metrics['Recall'], ridge_metrics['F1-Score'], ridge_metrics['ROC-AUC']],
    'LSTM': [lstm_metrics['Accuracy'], lstm_metrics['Precision'], 
             lstm_metrics['Recall'], lstm_metrics['F1-Score'], lstm_metrics['ROC-AUC']],
}

comparison_df = pd.DataFrame(comparison_data).set_index('Metric')
comparison_df['Difference (LSTM - Ridge)'] = comparison_df['LSTM'] - comparison_df['Ridge']
comparison_df['Winner'] = comparison_df['Difference (LSTM - Ridge)'].apply(
    lambda x: 'LSTM' if x > 0 else ('Ridge' if x < 0 else 'Tie')
)

print(comparison_df.round(4).to_string())

print("\n" + "="*70)

# --- 7. SAVE METRICS TO CSV ---
output_file = '../data/processed/06_model_metrics.csv'
metrics_df.to_csv(output_file)
print(f"\n✅ Metrics saved to {output_file}\n")

Loading model predictions...
Loaded 2330 predictions

MODEL EFFICIENCY METRICS

       Accuracy  Precision  Recall  F1-Score  ROC-AUC
Model                                                
Ridge    0.5069     0.5106  0.4910    0.5007   0.5070
LSTM     0.4996     0.5027  0.5473    0.5241   0.5049


RANKINGS BY METRIC:

Accuracy:
  1. Ridge              → 0.5069
  2. LSTM               → 0.4996

Precision:
  1. Ridge              → 0.5106
  2. LSTM               → 0.5027

Recall:
  1. LSTM               → 0.5473
  2. Ridge              → 0.4910

F1-Score:
  1. LSTM               → 0.5241
  2. Ridge              → 0.5007

ROC-AUC:
  1. Ridge              → 0.5070
  2. LSTM               → 0.5049


🏆 Best Overall Model (Average Rank): Ridge

   Metrics:
   - Accuracy:  0.5069
   - Precision: 0.5106
   - Recall:    0.4910
   - F1-Score:  0.5007
   - ROC-AUC:   0.5070


MODEL COMPARISON:

            Ridge    LSTM  Difference (LSTM - Ridge) Winner
Metric                                       